## Purpose

The purpose of this notebook is to document my full application of the Stacks pipeline on my cod data. I've been writing scripts to work on a subset of my files, and now I want to start running them on all of my files.

### ``process_radtags`` 

I don't have to run this on my data because the previous user of the data already did. In my script for running ``process_radtags``, however, I also use this opportunity to make folders for the output of each of the Stacks programs. So I'm doing that manually here. Below, you can see these directories plus two folders of cod data (two lanes of data):

![image](https://github.com/nclowell/FISH546/blob/master/Cod-Time-Series-Project/Notebooks/images_for_notebooks/Screen%20Shot%202016-11-16%20at%203.10.45%20PM.png?raw=true)

### ``ustacks`` on all of my files

I'll be working off of my hard drive, so first I had to change directory to there.

In [5]:
cd /Volumes/Time\ Machine\ Backups/Cod-Time-Series-Data/ 

/Volumes/Time Machine Backups/Cod-Time-Series-Data


Another issue is that I have two lanes. It'd be easiest if I could work in only one directory, but their barcodes are redundant. So my first step is to rename the files by adding *_L1* for Library 1 and *_L2* for Library 2. I'll do that in python before the file extension. 

Here's the content of the script ``add_lib_to_filename.py``:

```
# arguments
# [1] directory name with files to rename 
# [2] text you would like to add to the end of each filename before file extensions

# assumptions
# [1] file extention = fq.qz
# [2] you are one directory above the directory that you want to rename the files of

import sys
import subprocess

# make a text file that has each thing in this directory on a line; each line should be each file in the lane

string1 = "" # initiate string

string1 += 'ls ' + sys.argv[1] + ' > ./dircontents.txt' # add ls, wd, redirect to text file

firstshell = open("getcontents_shell.txt", "w")
firstshell.write(string1)
firstshell.close()

subprocess.call(["sh getcontents_shell.txt"], shell = True)

# now that file is made for contents of directory, read that file in
contents = open("dircontents.txt", "r")


# second shell = renaming script
rename_w_lib_shell = open("rename_w_lib_shell.txt", "w")

string2 = ""
string2 += "cd " + sys.argv[1] + "\n"


for line in contents:
	original = line.strip()
	filename_list = line.strip().split(".")
	filename_wo_ext = filename_list[0]
	filename = filename_wo_ext + sys.argv[2] + "." + filename_list[1] + "." + filename_list[2]
	string2 += "mv" + "\t" + original + "\t" + filename + "\n"

rename_w_lib_shell.write(string2)
rename_w_lib_shell.close()

# call renaming shell script
subprocess.call(["sh rename_w_lib_shell.txt"], shell = True)
```

Running the code looks like this:

In [8]:
!python add_lib_to_filename.py process_radtags_out/cod_lib1 _L1

In [9]:
!python add_lib_to_filename.py process_radtags_out/cod_lib2 _L2

Now my files are renamed with the library information after the barcode, so that I have no redundant file names.